# 🤖 Let's Go Robot! - Dobot MG400

Notebook สำหรับควบคุมแขนกล Dobot MG400 ผ่าน TCP/IP

**Features:**
- ใช้ค่า Z แบบ Online mode ได้เลย (แปลงเป็น TCP อัตโนมัติ)
- Z = -64 คือระดับพื้น
- รองรับ Homography Matrix สำหรับแปลง pixel → robot coordinates

## 1️⃣ Install Dependencies

In [ ]:
!pip install pydobot matplotlib opencv-python numpy

## 2️⃣ Imports & Setup

In [1]:
import cv2
import numpy as np
import time
import socket
import math
import importlib

# Reload config
import config
importlib.reload(config)

print("✓ Imports loaded")
print(f"Device: {config.DEVICE}")

[Config] Mode: ROBOT, Capture: FRAME_BY_FRAME, Device: cpu
[Config] Mode: ROBOT, Capture: FRAME_BY_FRAME, Device: cpu
✓ Imports loaded
Device: cpu


## 3️⃣ DobotControllerTCP Class

In [2]:
# =================================================================
# DobotControllerTCP Class พร้อม Z Offset (Online → TCP)
# =================================================================
# Online mode Z = -64 (ชิดพื้น) → TCP mode Z = -116
# Offset = 52 (TCP_Z = Online_Z - 52)
# =================================================================

class DobotControllerTCP:
    def __init__(self, homography_matrix=None):
        self.dashboard_port = 29999
        self.sock_dashboard = None
        self.homography_matrix = homography_matrix
        
        # ค่าจาก config (Online mode)
        self.safe_height = config.ROBOT_SAFE_HEIGHT  # -30
        self.grasp_height = config.ROBOT_GRASP_HEIGHT  # -64
        self.drop_pos = config.ROBOT_DROP_POS  # (250, 0, -30, 0)
        
        # Limits & Offset
        self.Z_MIN = config.ROBOT_Z_MIN  # -64
        self.Z_OFFSET = config.ROBOT_Z_OFFSET_TCP  # 52
        
        # Safety radius
        self.MIN_RADIUS = 180
        self.MAX_RADIUS = 450
        
    def online_to_tcp(self, z):
        """แปลง Z: Online mode → TCP mode"""
        return z - self.Z_OFFSET
        
    def connect(self, ip):
        try:
            self.sock_dashboard = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock_dashboard.settimeout(5)
            self.sock_dashboard.connect((ip, self.dashboard_port))
            
            print("Clearing Errors...")
            self.send_command("ClearError()") 
            time.sleep(0.5)
            
            print("Enabling Robot...")
            self.send_command("EnableRobot()") 
            time.sleep(4)
            
            self.send_command("User(0)")
            self.send_command("Tool(0)")
            self.send_command("SpeedFactor(50)")
            
            print(f"\n✅ Connected!")
            print(f"   Z Offset: {self.Z_OFFSET}")
            print(f"   Grasp Z: {self.grasp_height} (Online) → {self.online_to_tcp(self.grasp_height)} (TCP)")
            return True
        except Exception as e:
            print(f"Connection Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock_dashboard:
            try:
                self.sock_dashboard.send((cmd + "\n").encode("utf-8"))
                return self.sock_dashboard.recv(1024).decode("utf-8")
            except Exception as e:
                print(f"Command Error: {e}")
        return None

    def home(self):
        print("🤖 Moving to HOME...")
        self.send_command("JointMovJ(0,0,0,0)")
        time.sleep(4)
        print("✅ HOME")

    def toggle_suction(self, state):
        self.send_command(f"DO(1,{1 if state else 0})") 
        time.sleep(0.5)

    def move_z(self, x, y, z_online, r, movj=False):
        """เคลื่อนที่ - ใส่ Z แบบ Online mode (แปลงให้อัตโนมัติ)"""
        if z_online < self.Z_MIN:
            print(f"⚠️ Z={z_online} below limit! Clamped to {self.Z_MIN}")
            z_online = self.Z_MIN
        
        z_tcp = self.online_to_tcp(z_online)
        cmd = f"MovJ({x},{y},{z_tcp},{r})" if movj else f"MovL({x},{y},{z_tcp},{r})"
        print(f"   Z: {z_online} (Online) → {z_tcp} (TCP)")
        return self.send_command(cmd)

    def transform_pixel_to_robot(self, u, v):
        """แปลง pixel → robot coordinates"""
        if self.homography_matrix is None:
            return None, None
        point = np.array([u, v, 1], dtype=np.float32)
        result = np.dot(self.homography_matrix, point)
        return result[0] / result[2], result[1] / result[2]

    def set_heights(self, safe_h, grasp_h):
        self.safe_height = safe_h
        self.grasp_height = grasp_h
        print(f"✅ Heights: Safe={safe_h}, Grasp={grasp_h}")

    def set_drop_position(self, x, y, z, r):
        self.drop_pos = (x, y, z, r)

print("✓ DobotControllerTCP class loaded")

✓ DobotControllerTCP class loaded


## 4️⃣ Robot Configuration

In [22]:
# =================================================================
# Robot Configuration
# =================================================================

# Robot IP
ROBOT_PORT = '192.168.1.6'

# Homography Matrix (จาก calibrate.py)
HOMOGRAPHY_MATRIX = np.array([
    [-1.0372325800136264, -0.020282799398624013, 264.7854562980147],
    [0.027826479833018232, 1.0278958788918955, -283.03518239459316],
    [0.0010485712551503673, -0.0003729007208423201, 1.0],
], dtype=np.float32)

print("✓ Configuration loaded")
print(f"  Robot IP: {ROBOT_PORT}")
print(f"  Grasp Z: {config.ROBOT_GRASP_HEIGHT} (Online)")
print(f"  Safe Z: {config.ROBOT_SAFE_HEIGHT} (Online)")

✓ Configuration loaded
  Robot IP: 192.168.1.6
  Grasp Z: -62 (Online)
  Safe Z: -30 (Online)


## 5️⃣ Connect to Robot

In [35]:
# =================================================================
# Connect to Robot
# =================================================================

robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX)



if robot.connect(ROBOT_PORT):
    print("\n" + "="*60)
    print(f"🤖 Connected to {ROBOT_PORT}!")
    print("="*60)
else:
    print("⚠️ Failed to connect")

Clearing Errors...
Enabling Robot...

✅ Connected!
   Z Offset: 52
   Grasp Z: -62 (Online) → -114 (TCP)

🤖 Connected to 192.168.1.6!


In [37]:
# แก้ในเซลล์ Connect หรือรันเซลล์นี้หลัง connect

robot.send_command("User(1)")  # เปลี่ยนจาก 0 เป็น 1
robot.send_command("Tool(1)")  # เปลี่ยนจาก 0 เป็น 1

print("✅ Changed to User(1), Tool(1)")

✅ Changed to User(1), Tool(1)


## 6️⃣ Home Robot


In [53]:
# =================================================================
# Home Robot
# =================================================================

print("⚠️ Robot will move to HOME!")
if input("Confirm? (y/n): ").lower() == 'y':
    robot.home()

⚠️ Robot will move to HOME!
🤖 Moving to HOME...
✅ HOME


## 7️⃣ Test Calibration (A, B, C, D)

In [ ]:
# ลองเคลื่อน พิกัดสี่เหลี่ยมจตุรัส 50x50 mm ตำแหน่งปลอดภัย
# Z = -64 (Online mode = ชิดพื้น)
test_points = [
    ("A", 300, -25),   # มุมบนซ้าย
    ("B", 300, 25),    # มุมบนขวา
    ("C", 350, 25),    # มุมล่างขวา
    ("D", 350, -25),   # มุมล่างซ้าย
]

Z_HEIGHT = -64  # Online mode
R_ANGLE = 0

print("🤖 เริ่มทดสอบวิ่งสี่เหลี่ยมจตุรัส")
print("=" * 50)

for name, x, y in test_points:
    print(f"\n📍 เคลื่อนไปจุด {name}: X={x}, Y={y}, Z={Z_HEIGHT}")
    robot.move_z(x, y, Z_HEIGHT, R_ANGLE, movj=True)
    time.sleep(2)
    print(f"✅ ถึงจุด {name} แล้ว")

# กลับจุดเริ่มต้น A
print(f"\n🔄 กลับจุด A")
robot.move_z(test_points[0][1], test_points[0][2], Z_HEIGHT, R_ANGLE, movj=True)

print("\n" + "=" * 50)
print("✅ ทดสอบเสร็จสิ้น!")

In [ ]:
# วิ่งอัตโนมัติ 4 จุด
calibration_points = [
    ("A", 86.47, 16.14),
    ("B", 10.26, 16.14),
    ("C", 89.96, 91.62),
    ("D", 10.26, 91.62),
]

Z = -64
R = 0
DELAY = 3  # วินาที

print("🎯 Auto run: A → B → C → D → A")
if input("Confirm? (y/n): ").lower() == 'y':
    for name, x, y in calibration_points:
        print(f"📍 {name}: ({x}, {y})")
        robot.send_command(f"MovJ({x},{y},{Z},{R})")
        time.sleep(DELAY)
    
    # กลับจุด A
    print("🔄 Back to A")
    robot.send_command(f"MovJ({calibration_points[0][1]},{calibration_points[0][2]},{Z},{R})")
    time.sleep(DELAY)
    
    print("✅ Complete!")

🎯 Auto run: A → B → C → D → A
📍 A: (86.47, 16.14)
📍 B: (10.26, 16.14)
📍 C: (89.96, 91.62)
📍 D: (10.26, 91.62)
🔄 Back to A
✅ Complete!


## 8️⃣ Manual Grasp Test

In [ ]:
# =================================================================
# Manual Grasp Test
# =================================================================

# ใส่ค่า Online mode ได้เลย!
X, Y = 309, 40
GRASP_Z = -64  # ชิดพื้น
SAFE_Z = -30   # ยกขึ้น

print(f"Target: ({X}, {Y}), Grasp Z={GRASP_Z}")

if input("Confirm? (y/n): ").lower() == 'y':
    print("1. Approach...")
    robot.move_z(X, Y, SAFE_Z, 0, movj=True)
    time.sleep(3)
    
    print("2. Descend...")
    robot.move_z(X, Y, GRASP_Z, 0)
    time.sleep(2)
    
    print("3. Suction ON...")
    robot.toggle_suction(True)
    time.sleep(1)
    
    print("4. Lift...")
    robot.move_z(X, Y, SAFE_Z, 0)
    time.sleep(2)
    
    print("5. Drop...")
    dx, dy, dz, dr = robot.drop_pos
    robot.move_z(dx, dy, dz, dr, movj=True)
    time.sleep(4)
    
    print("6. Release...")
    robot.toggle_suction(False)
    
    print("🎉 Done!")